# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../Resources/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,rikaze,29.2500,88.8833,275.87,42,88,1.97,CN,1712271082
1,1,ilulissat,69.2167,-51.1000,269.16,93,75,2.06,GL,1712271082
2,2,mesolongi,38.3702,21.4295,288.66,71,0,1.39,GR,1712271082
3,3,adamstown,-25.0660,-130.1015,295.82,62,100,4.36,PN,1712271082
4,4,udachny,66.4167,112.4000,250.07,95,100,12.48,RU,1712271083


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng',  
    'Lat',  
    geo=True, 
    size='Humidity'
)

# Display the map
map_plot


:Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values

# Narrow down cities that fit criteria
filtered_cities_df = city_data_df[(city_data_df['Cloudiness'] <= 20) & (city_data_df['Wind Speed'] >= 1) & (city_data_df['Wind Speed'] <= 5)]

# Drop any rows with null values
#.dropna() found using Chat GPT
filtered_cities_df = filtered_cities_df.dropna()

# Display sample data
filtered_cities_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,mesolongi,38.3702,21.4295,288.66,71,0,1.39,GR,1712271082
13,13,macas,-2.3167,-78.1167,301.01,57,20,4.12,EC,1712270913
14,14,yuzhno-kurilsk,44.0306,145.8556,273.91,98,15,2.53,RU,1712271084
17,17,seydisehir,37.4193,31.8453,283.74,55,4,1.23,TR,1712271084
19,19,happy valley-goose bay,53.3168,-60.3315,276.32,56,20,3.09,CA,1712271085
...,...,...,...,...,...,...,...,...,...,...
547,547,darab,28.7519,54.5444,290.61,45,9,1.13,IR,1712271146
548,548,akureyri,65.6835,-18.0878,264.11,73,0,2.06,IS,1712271146
556,556,san pedro pochutla,15.7333,-96.4667,302.99,56,19,3.44,MX,1712271147
558,558,alice springs,-23.7000,133.8833,291.60,43,1,3.60,AU,1712271106


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
0,rikaze,CN,29.2500,88.8833,42,
1,ilulissat,GL,69.2167,-51.1000,93,
2,mesolongi,GR,38.3702,21.4295,71,
3,adamstown,PN,-25.0660,-130.1015,62,
4,udachny,RU,66.4167,112.4000,95,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
# Set parameters to search for a hotel
limit = 20
radius = 10000
categories= "accommodation.hotel"
params = {
    "limit":limit,
    "apiKey":geoapify_key,
    "categories":categories
}
# Print a message to follow up the hotel search
print("Starting hotel search")
# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    Latitude = row["Lat"]
    Longitude = row["Lng"]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    # params["categories"] = f"accommodation.hotel"
    params["filter"] = f"circle:{Longitude},{Latitude},{radius}"
    params["bias"] = f"proximity:{Longitude},{Latitude}"
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    # Convert the API response to JSON format
    name_address = name_address.json()
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
# Display sample data
hotel_df

Starting hotel search
rikaze - nearest hotel: Gesar Hotel
ilulissat - nearest hotel: Best Western Plus Hotel Ilulissat
mesolongi - nearest hotel: AVRA
adamstown - nearest hotel: No hotel found
udachny - nearest hotel: Вилюй
port-aux-francais - nearest hotel: Keravel
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
utrik - nearest hotel: No hotel found
ivatsevichy - nearest hotel: Івацэвічы
pisco - nearest hotel: La Portada
sicuani - nearest hotel: Hostal El Imperador
waitangi - nearest hotel: Hotel Chathams
macas - nearest hotel: Nivel 5
yuzhno-kurilsk - nearest hotel: Айсберг
mount isa - nearest hotel: Ibis Styles
albertina - nearest hotel: Hotel Parque das Primaveras
seydisehir - nearest hotel: sey-han hotel
san antonio de pale - nearest hotel: No hotel found
happy valley-goose bay - nearest hotel: Hotel North Two
aasiaat - nearest hotel: SØMA
touros - nearest hotel: Pousada Atlântico
east london 

,City,Country,Lat,Lng,Humidity,Hotel Name
0,rikaze,CN,29.2500,88.8833,42,Gesar Hotel
1,ilulissat,GL,69.2167,-51.1000,93,Best Western Plus Hotel Ilulissat
2,mesolongi,GR,38.3702,21.4295,71,AVRA
3,adamstown,PN,-25.0660,-130.1015,62,No hotel found
4,udachny,RU,66.4167,112.4000,95,Вилюй
...,...,...,...,...,...,...
566,greenville,US,35.6127,-77.3663,40,Quality Inn
567,chernyshevskiy,RU,63.0128,112.4714,88,No hotel found
568,mont-dore,NC,-22.2833,166.5833,99,Les Cases de Plum
569,brownfield,US,33.1812,-102.2744,10,Best Western Caprock Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
%%capture --no-display

# Configure the map plot with hover information
map_plot_with_hotels = hotel_df.hvplot.points(
    'Lng',  
    'Lat',  
    geo=True, 
    size='Humidity',
    hover_cols=['City', 'Country', 'Hotel Name', 'Humidity']  
)

# Display the map
map_plot_with_hotels


:Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)